$\newcommand{\xbf}{{\bf x}}
\newcommand{\ybf}{{\bf y}}
\newcommand{\wbf}{{\bf w}}
\newcommand{\Ibf}{\mathbf{I}}
\newcommand{\Xbf}{\mathbf{X}}
\newcommand{\Rbb}{\mathbb{R}}
\newcommand{\vec}[1]{\left[\begin{array}{c}#1\end{array}\right]}
$

# Introduction aux réseaux de neurones : TD #2  (partie 1)
Matériel de cours rédigé par Pascal Germain, 2018
************

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import torch
from torch import nn

### Initialisation «Xavier» et «Kaiming»

In [ ]:
nn.init.xavier_normal_?

In [ ]:
L = nn.Linear(5,5)
L.weight

In [ ]:
nn.init.xavier_uniform_(L.weight)

In [ ]:
L.weight

In [ ]:
nn.init.kaiming_normal_(L.weight)

In [ ]:
L.weight

### Dropout

In [ ]:
nn.Dropout?

In [ ]:
drop = nn.Dropout(p=.5)

In [ ]:
v = torch.ones(10)
drop(v)

In [ ]:
for i in range(10):
    print(drop(v))

In [ ]:
drop = nn.Dropout(p=.75)
for i in range(10):
    print(drop(v))

In [ ]:
drop.eval?

In [ ]:
drop.eval() # Mode prédiction
for i in range(10):
    print(drop(v))

In [ ]:
drop.train() # Mode apprentissage
for i in range(10):
    print(drop(v))

In [ ]:
model = nn.Sequential(nn.Linear(3,5),
                      nn.Tanh(),
                      nn.Dropout())

In [ ]:
x = torch.rand(3)
x

In [ ]:
for i in range(10):
    print(model(x))

In [ ]:
model.eval()
model(x)

In [ ]:
model.train()
model(x)

### Apprentissage par «minibatch»

In [ ]:
from torch.utils.data import TensorDataset, DataLoader

In [ ]:
X = (torch.arange(1, 11, dtype=torch.float32) * torch.ones(3,10, dtype=torch.float32)).transpose(0,1)
X

In [ ]:
Y = 10 * torch.arange(1, 11) 
Y

In [ ]:
data = TensorDataset(X,Y)

In [ ]:
data[2]

In [ ]:
data[2:5]

In [ ]:
for x,y in data:
    print(x, '<-->', y)

In [ ]:
DataLoader?

In [ ]:
sampler = DataLoader(data, batch_size=3)
for t in range(4):
    print('******* EPOQUE', t)
    for x, y in sampler:
        print(x, '<-->', y)

In [ ]:
sampler = DataLoader(data, batch_size=3, shuffle=True)
for t in range(4):
    print('******* EPOQUE', t)
    for x, y in sampler:
        print(x, '<-->', y)

### Normalisation «batchnorm»

Considérons un neurone. Lors de la phase de propagation de l'apprentissage, ce neurone est «traversé» par une minibatch de $m$ éléments au «temps» $t$.
1.  Notons $a_{t;1}, \ldots, a_{t;m}$ l'ensemble des valeurs sortie du neurone.
2. On calcule 
$$\begin{align*}
\mu_{a_t} &= \frac1m \sum_{i=1}^m a_{t;i}\\
\sigma_{a_t}  &= \sqrt{\epsilon + \frac1m(\mu_{a_t}  - a_{t;i} )^2} \qquad \mbox{($\epsilon\approx 10^{-8}$)}
\end{align*}$$
3. Puis, on transforme chaque $a_i$:
$$\begin{align*}\tilde a_k &= \gamma_a \frac{a_{t;i}-\mu_{a_t} }{\sigma_{a_t} } + \beta_a,\end{align*}$$
où $\gamma_a$ et $\beta_a$ sont des paramètres associés au neurone au même titre que les poids, initialisés au hasard et appris par rétropropagation du gradient.

$\Rightarrow$ Lors de la prédiction, $\gamma_a$ et $\beta)a$ sont remplacés par des valeurs estimés sur tout l'ensemble d'apprentissage.


nn.BatchNorm1d?

In [ ]:
bn = nn.BatchNorm1d(3)

In [ ]:
bn.weight # correspond au paramètre gamma_a

In [ ]:
bn.bias # correspond au paramètre beta_a

In [ ]:
sampler = DataLoader(data, batch_size=2, shuffle=False)
for x, y in sampler:
    print(x)
    print(bn(x))
    print('---')

In [ ]:
model = nn.Sequential(nn.Linear(3,5),
                      nn.Tanh(),
                      nn.BatchNorm1d(5))

In [ ]:
model.train() # Mode apprentissage
model(X)

In [ ]:
model.eval() # Mode prédiction
model(X)

### Softmax

In [ ]:
nn.Softmax?

In [ ]:
a = torch.randn(5)

In [ ]:
sm = nn.Softmax(dim=0)
sm(a)

In [ ]:
torch.sum(sm(a))

Pour des raisons de stabilité numérique, *pyTorch* préfère travailler avec le logarithme de l'activation SoftMax

In [ ]:
nn.LogSoftmax?

In [ ]:
logsm = nn.LogSoftmax(dim=0) # Il  faut spécifier la dimension selon laquelle appliquer la normalisation

In [ ]:
logsm(a)

In [ ]:
torch.log(sm(a))

In [ ]:
model = nn.Sequential(
                nn.Linear(10, 3),
                nn.LogSoftmax(dim=1) # Normalise chaque ligne
                )

In [ ]:
X = torch.randn(5, 10)
X

In [ ]:
pred = model(X)
pred

In [ ]:
pred.exp().sum(dim=1)

In [ ]:
pred.argmax(dim=1)

#### Perte du néfatif log vraissemblance associée à une sortie softmax

Pour nous faciliter la tâche, la fonction de perte `NLLLoss` est conçue pour gérer une paire d'arguments. Pour une minibatch de taille $m$ et un problème à $C$ classes, le calcul de la perte se fait à partir de:
1. La prédiction donnée par une activation `LogSoftmax` (de taille $m\times C$)
2. La sortie désirée sous la forme d'un vecteur de $m$ éléments, chacun de ces éléments étant un entier de $0$ à $C-1$

In [ ]:
nn.NLLLoss?

In [ ]:
perte = nn.NLLLoss()

In [ ]:
Y = torch.ones(5, dtype=torch.int64)
perte(pred, Y)